In [1]:
from cval import CVal

config_file = "../config.toml"
env_file = "../.env"

cval = CVal.init(
    config_file=config_file,
    env_file=env_file
)

/home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


[06/25/24 13:08:38] INFO     Ingestion engine initialized.                                          ]8;id=430119;file:///home/simisimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=750371;file:///home/simisimon/GitHub/cval/src/ingestion.py#42\42]8;;\

                    INFO     Retrieval engine initialized.                                          ]8;id=764420;file:///home/simisimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=75247;file:///home/simisimon/GitHub/cval/src/retrieval.py#29\29]8;;\

                    INFO     Ingestion engine initialized.                                          ]8;id=404350;file:///home/simisimon/GitHub/cval/src/ingestion.py\ingestion.py]8;;\:]8;id=704420;file:///home/simisimon/GitHub/cval/src/ingestion.py#42\42]8;;\

                    INFO     CVal initialized.                                                          ]8;id=525650;file:///home/simisimon/GitHub/cval/src/cval.py\cval.py]8;;\:]8;id=751119;file:///home/simisimon/GitHub/cval/src/cval.py#101\101]8;;\

In [2]:
import pandas as pd

eval_data_file = "../data/eval/dependencies_without_rules.csv"

df = pd.read_csv(eval_data_file, sep=";")

df_sample = df[:10]

df_sample

,link_str,project,option_name,option_value,option_file,option_type,option_technology,dependent_option_name,dependent_option_value,dependent_option_file,dependent_option_type,dependent_option_technology,pre_rating,rating
0,mall::::mall-search/pom.xml::::project::::depe...,mall,project.dependencies.dependency_mall-mbg.groupId,com.macro.mall,mall-search/pom.xml,ConfigType.NAME,maven,project.parent_mall.groupId,com.macro.mall,mall-security/pom.xml,ConfigType.NAME,maven,False,Unsure
1,mall::::mall-demo/pom.xml::::project::::depend...,mall,project.dependencies.dependency_spring-boot-st...,org.springframework.boot,mall-demo/pom.xml,ConfigType.NAME,maven,project.dependencies.dependency_spring-boot-st...,org.springframework.boot,mall-portal/pom.xml,ConfigType.NAME,maven,True,True
2,mall::::mall-admin/src/main/resources/applicat...,mall,spring.datasource.druid.stat-view-servlet.logi...,druid,mall-admin/src/main/resources/application-dev.yml,ConfigType.VERSION_NUMBER,spring,spring.datasource.druid.stat-view-servlet.logi...,druid,mall-search/src/main/resources/application-pro...,ConfigType.VERSION_NUMBER,spring,True,True
3,mall::::mall-mbg/pom.xml::::project::::groupId...,mall,project.groupId,com.macro.mall,mall-mbg/pom.xml,ConfigType.NAME,maven,project.parent_mall.groupId,com.macro.mall,mall-mbg/pom.xml,ConfigType.NAME,maven,True,True
4,mall::::mall-portal/pom.xml::::project::::depe...,mall,project.dependencies.dependency_spring-boot-st...,org.springframework.boot,mall-portal/pom.xml,ConfigType.NAME,maven,project.dependencies.dependency_spring-boot-co...,org.springframework.boot,pom.xml,ConfigType.NAME,maven,True,True
5,apollo::::apollo-biz/src/test/resources/applic...,apollo,spring.h2.console.enabled,true,apollo-biz/src/test/resources/application.prop...,ConfigType.BOOLEAN,spring,project.build.plugins.plugin_maven-jar-plugin....,true,pom.xml,ConfigType.IP_ADDRESS,maven,False,False
6,apollo::::apollo-portal/pom.xml::::project::::...,apollo,project.dependencies.dependency_apollo-openapi...,com.ctrip.framework.apollo,apollo-portal/pom.xml,ConfigType.NAME,maven,project.dependencyManagement.dependencies.depe...,com.ctrip.framework.apollo,pom.xml,ConfigType.NAME,maven,False,True
7,apollo::::apollo-assembly/pom.xml::::project::...,apollo,project.dependencies.dependency_apollo-portal....,apollo-portal,apollo-assembly/pom.xml,ConfigType.NAME,maven,COPY.src,apollo-portal,apollo-portal/src/main/docker/Dockerfile,ConfigType.PATH,docker,False,False
8,apollo::::apollo-assembly/pom.xml::::project::...,apollo,project.parent_apollo.relativePath,../pom.xml,apollo-assembly/pom.xml,ConfigType.PATH,maven,project.parent_apollo.relativePath,../pom.xml,apollo-portal/pom.xml,ConfigType.PATH,maven,NaN,True
9,apollo::::apollo-portal/src/test/resources/app...,apollo,spring.h2.console.enabled,true,apollo-portal/src/test/resources/application.p...,ConfigType.BOOLEAN,spring,project.build.plugins.plugin_maven-jar-plugin....,true,pom.xml,ConfigType.IP_ADDRESS,maven,False,False


In [3]:
from deepeval.metrics import AnswerRelevancyMetric, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase
from dotenv import dotenv_values
import os

config = dotenv_values(dotenv_path="../.env")
os.environ['OPENAI_API_KEY'] = config["OPENAI_KEY"]


from deepeval.models.base_model import DeepEvalBaseLLM
from llama_index.llms.ollama import Ollama

model = Ollama(model="llama3:8b", temperature=0.0)

class LlamaModel(DeepEvalBaseLLM):
    def __init__(
        self,
        model
    ):
        self.model = model

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()
        return model.complete(prompt).text

    async def a_generate(self, prompt: str) -> str:
        model = self.load_model()
        res = await model.acomplete(prompt)
        return res.text

    def get_model_name(self):
        return "Custom Llama3 model"


answer_relevancy_metric = AnswerRelevancyMetric(
    threshold=0.7,
    model="gpt-4",
    include_reason=True
)

faithfulness_metric = FaithfulnessMetric(
    threshold=0.7,
    model="gpt-4",
    include_reason=True
)

context_relevancy_metric = ContextualRelevancyMetric(
    threshold=0.7,
    model="gpt-4",
    include_reason=True
)

/home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/deepeval/__init__.py:42: UserWarning: You are using deepeval version 0.21.60, however version 0.21.61 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


In [4]:
from data import Dependency
import mlflow
import datetime
import json

index_name = "web-search"
exp_name = "dependencies_without_rules"
date_time = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")

mlflow.set_experiment(experiment_name=exp_name)

with mlflow.start_run(run_name=f"{exp_name}_{date_time}"):

    output = []

    for x in df_sample.to_dict("records"):
        dependency = Dependency(
            project=x["project"],
            option_name=x["option_name"],
            option_value=x["option_value"],
            option_type=x["option_type"].split(".")[-1],
            option_file=x["option_file"],
            option_technology=x["option_technology"],
            dependent_option_name=x["dependent_option_name"],
            dependent_option_value=x["dependent_option_value"],
            dependent_option_type=x["dependent_option_type"].split(".")[-1],
            dependent_option_file=x["dependent_option_file"],
            dependent_option_technology=x["dependent_option_technology"]
        )

        response = cval.query(
            dependency=dependency,
            index_name=index_name
        )

        output.append(response)

    inputs = [x.input for x in output]
    results = [x.response for x in output]
    mlflow.log_table(data={"inputs": inputs, "outputs": results}, artifact_file="results.json")

    

[06/25/24 13:08:40] INFO     Select index: github.                                                  ]8;id=675081;file:///home/simisimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=506405;file:///home/simisimon/GitHub/cval/src/retrieval.py#47\47]8;;\

[06/25/24 13:08:45] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=235127;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=887775;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

[06/25/24 13:08:46] INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200  ]8;id=958713;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=246887;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

[06/25/24 13:08:47] INFO     Len of retrieved nodes of query 0: 5                                  ]8;id=445351;file:///home/simisimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=428042;file:///home/simisimon/GitHub/cval/src/retrieval.py#174\174]8;;\

                    INFO     HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200  ]8;id=590297;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=520483;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             OK"                                                                                   

...

[06/25/24 13:12:47] INFO     Len of retrieved nodes after reranking: 3                             ]8;id=402;file:///home/simisimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=818370;file:///home/simisimon/GitHub/cval/src/retrieval.py#117\117]8;;\

Batches: 100%|██████████| 1/1 [00:00<00:00,  5.26it/s]


[06/25/24 13:12:48] INFO     Len of retrieved nodes after reranking: 3                             ]8;id=234215;file:///home/simisimon/GitHub/cval/src/retrieval.py\retrieval.py]8;;\:]8;id=745229;file:///home/simisimon/GitHub/cval/src/retrieval.py#117\117]8;;\

                    INFO     GPT (gpt-4o-2024-05-13) generator initialized.                         ]8;id=167220;file:///home/simisimon/GitHub/cval/src/generator.py\generator.py]8;;\:]8;id=61284;file:///home/simisimon/GitHub/cval/src/generator.py#46\46]8;;\

[06/25/24 13:12:55] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=567669;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=541286;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

In [5]:
faithfulness_scores = []
answer_relevancy_scores = []
context_relevancy_scores = []

false_positives = []
true_positives = []
false_negatives = []
true_negatives = []
accuracy = []

for response, baseline in zip(output, df_sample.to_dict("records")):


    test_case = LLMTestCase(
        input=response.input,
        actual_output=response.response,
        retrieval_context=[source_node.node.get_content() for source_node in response.source_nodes]
    )

    context_relevancy_metric.measure(test_case)
    faithfulness_metric.measure(test_case)
    answer_relevancy_metric.measure(test_case)

    context_relevancy_scores.append(context_relevancy_metric.score)
    faithfulness_scores.append(faithfulness_metric.score)
    answer_relevancy_scores.append(answer_relevancy_metric.score)       
            
    response_dict = json.loads(response.response)
        
    print("Context Relevancy: ", context_relevancy_scores, sum(context_relevancy_scores)/len(context_relevancy_scores))
    print("Answer Relevancy: ", answer_relevancy_scores, sum(answer_relevancy_scores)/len(answer_relevancy_scores))
    print("Faithfulness: ", faithfulness_scores, sum(faithfulness_scores)/len(faithfulness_scores))

/home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Event loop is already running. Applying nest_asyncio patch to allow async execution...

/home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[06/25/24 13:12:56] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=423030;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=201709;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             "HTTP/1.1 200 OK"                                                                     

                    INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=988612;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=487117;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             "HTTP/1.1 200 OK"                                                                     

...

/home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[06/25/24 13:22:30] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=901022;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=15815;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             "HTTP/1.1 200 OK"                                                                     

/home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" for
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[06/25/24 13:22:33] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=358609;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=245073;file:///home/simisimon/GitHub/cval/env/lib/python3.9/site-packages/httpx/_client.py#1773\1773]8;;\
                             "HTTP/1.1 200 OK"                                                                     

Context Relevancy:  [0.6666666666666666, 0.0, 1.0, 1.0, 0.0, 0.0, 0.3333333333333333, 0.0, 0.6666666666666666, 0.0] 0.36666666666666664
Answer Relevancy:  [0.8181818181818182, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.8461538461538461, 1.0, 1.0] 0.9664335664335664
Faithfulness:  [1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 1.0, 1.0, 1.0, 1.0] 0.95


In [6]:
true_positives = []
true_negatives = []
false_positives = []
false_negatives = []
accuracy = []

for response, baseline in zip(output, df_sample.to_dict("records")):
    response_dict = json.loads(response.response)


    if baseline["rating"] in ("True", "Unsure") and response_dict["isDependency"]:
        accuracy.append(1)
        true_positives.append(1)
    if baseline["rating"] in ("False") and not response_dict["isDependency"]:
        accuracy.append(1)
        true_negatives.append(1)
    if baseline["rating"] in ("True", "Unsure") and not response_dict["isDependency"]:
        accuracy.append(0)
        false_negatives.append(1)
    if baseline["rating"] in ("False") and response_dict["isDependency"]:
        accuracy.append(0)
        false_positives.append(1)


#precision = sum(true_positives)/(sum(true_positives)+sum(false_positives))
#recall = sum(true_positives)/(sum(true_positives)+sum(false_negatives))
#f1_score = 2 * (precision * recall) / (precision + recall)

print("Accuracy", sum(accuracy)/len(accuracy))
print("TP", sum(true_positives))
print("FP", sum(false_positives))
print("TN", sum(true_negatives))
print("FN", sum(false_negatives))
#print("Precision", precision)
#print("Recall", recall)
#print("F1 Score: ", f1_score)

Accuracy 0.5
TP 2
FP 0
TN 3
FN 5


In [7]:
#inputs = [x.input for x in output]
results = [x.response for x in output]
rating = df_sample["rating"]

df_results = pd.DataFrame().from_dict(data={"outputs": results, "isDependency": rating})
df_results.to_csv(f"../data/results/test_{index_name}.csv", index=False)


In [8]:
for x in output:
    print(x.input)
    print(x.response)
    break

You are a full-stack expert in validating intra-technology and cross-technology configuration dependencies.
You will be presented with configuration options found in the software project 'mall'.
Your task is to determine whether the given configuration options actually depend on each other based on value-equality.

A value-equality dependency is present if two configuration options must have identical values in order to function correctly.
Inconsistencies in these configuration values can lead to configuration errors. 
Importantly, configuration options may have equal values by accident, meaning that there is no actual dependency, but it just happens that they have equal values.
Information about both configuration options, including their descriptions and prior usages are stated below:
<?xml version="1.0" encoding="UTF-8"?>
<project xmlns="http://maven.apache.org/POM/4.0.0"
         xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
         xsi:schemaLocation="http://maven.apache.